Imports

In [1]:
import logging
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import warnings
warnings.filterwarnings("ignore")

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


Load Data <br>
Train-test split

In [ ]:
df = pd.read_parquet("D:\Productivity\Studying\PMLDL_A1\data\interim\\formatted.parquet")

df = df.sort_values(by="fit_score", ascending=False).reset_index(drop=True)

train_df = df[["reference", "translation"]].sample(frac=0.8, random_state=42)
eval_df = df[["reference", "translation"]].drop(train_df.index)

eval_df.reset_index(inplace=True, drop=True)
train_df.reset_index(inplace=True, drop=True)
eval_df.columns = ["input_text", "target_text"]
train_df.columns = ["input_text", "target_text"]

Set up model

In [10]:
model_args = Seq2SeqArgs()
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_seq_length = 128
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.use_early_stopping = True

model_args.do_sample = True
model_args.num_beams = 1
model_args.num_return_sequences = 1
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95
model_args.output_dir = "D:\Productivity\Studying\PMLDL_A1\models"

model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="eugenesiow/bart-paraphrase",
    args=model_args,
)

Train

In [11]:
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/374399 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/46800 [00:00<?, ?it/s]

  0%|          | 0/93600 [00:00<?, ?it/s]

  0%|          | 0/93600 [00:00<?, ?it/s]

  0%|          | 0/93600 [00:00<?, ?it/s]

  0%|          | 0/93600 [00:00<?, ?it/s]

(10000,
 {'global_step': [2500, 5000, 7500, 10000],
  'eval_loss': [0.21864687596140295,
   0.22097599173062726,
   0.2194783261080189,
   0.22354674145263592],
  'train_loss': [0.167571023106575,
   0.1507573127746582,
   0.19734515249729156,
   0.2759063243865967]})